In [2]:
###transformer_name = 'sentence-transformers/all-MiniLM-L6-v2'
embedding_model_name = "sentence-transformers/all-mpnet-base-v2" #open source?
llm_name = "databricks/dolly-v2-2-8b"

KB_doc_folder = "KnowledgeBaseDocs"
vector_db_path = "AIProjectDB/vector_db"

text_chunk_size_nchar = 800
chunk_overlap_size = 400
load = True
maxSemResCount = 50
recomputePlanCollections = False

In [3]:
if load:
    !pip -q install git+https://github.com/huggingface/transformers
    !pip -q install accelerate>=0.12.0
    !pip install langchain
    !pip install transformers
    !pip install sentence-transformers
    !pip install unstructured
    !pip install chromadb==0.3.22 transformers==4.29.0
    #!pip install chromadb-client
    !pip install pdf2image pdfminer
    !pip install pdfminer.six

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 17.4 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.2/383.2 kB 5.8 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 11.8 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 1.4 MB/s eta 0:00:00
  Attempting uninstall: numexpr
    Found existing installation: numexpr 2.7.3
    Uninstalling numexpr-2.7.3:
      Successfully uninstalled numexpr-2.7.3
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 12.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB

In [4]:
import os
from sentence_transformers import SentenceTransformer
from langchain.embeddings import HuggingFaceEmbeddings

###model = SentenceTransformer(transformer_name)
embed_model = HuggingFaceEmbeddings(model_name=embedding_model_name)

In [5]:
#import chromadb
#from chromadb.config import Settings
#client = chromadb.PersistentClient(settings=Settings(chroma_api_impl="chromadb.api.fastapi.FastAPI", persist_directory=vector_db_path))
#client = chromadb.Client(Settings(chroma_api_impl="chromadb.api.fastapi.FastAPI", persist_directory=vector_db_path, chroma_server_host='LocalHost', chroma_server_http_port=8000))
#client = chromadb.HttpClient(host='localhost', port=8000)
#client = chromadb.Client()

In [6]:
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os.path
from langchain.vectorstores import Chroma

# Don't recompute the embeddings if they're already available
#full_collection = client.create_collection(name="public_docs", embedding_function=embed_model)
db = Chroma(collection_name="public_docs", embedding_function=embed_model, persist_directory=vector_db_path)
if not os.path.isdir(vector_db_path) or len(os.listdir(vector_db_path)) == 0:
    if not os.path.isdir(vector_db_path):
      print(f"creating folder {vector_db_path}")
      !mkdir -p {vector_db_path}
    
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=text_chunk_size_nchar,chunk_overlap=chunk_overlap_size)
    for foldername in os.listdir(KB_doc_folder):
        folderpath = KB_doc_folder + "/" + foldername
        for filename in os.listdir(folderpath):
            filepath = folderpath + "/" + filename
            with open(filepath):
                loader = UnstructuredPDFLoader(filepath) #creating multiple times? look into params
                data = loader.load()
                text_chunks = text_splitter.split_documents(data)

                print(f"Saving document embeddings under {vector_db_path}")
                db = Chroma.from_documents(collection_name="public_docs", documents=text_chunks, embedding=embed_model, persist_directory=vector_db_path)
                db.persist()

Using embedded DuckDB with persistence: data will be stored in: AIProjectDB/vector_db


In [7]:
if recomputePlanCollections:
    if not os.path.isdir(vector_db_path):
      print(f"creating folder {vector_db_path}")
      !mkdir -p {vector_db_path}
    
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=text_chunk_size_nchar,chunk_overlap=chunk_overlap_size)
    for foldername in os.listdir(KB_doc_folder):
        folderpath = KB_doc_folder + "/" + foldername
        print(os.listdir(KB_doc_folder))
        for filename in os.listdir(folderpath):
            filepath = folderpath + "/" + filename
            with open(filepath):
                loader = UnstructuredPDFLoader(filepath) #creating multiple times? look into params
                data = loader.load()
                text_chunks = text_splitter.split_documents(data)
                #create a collection for each plan and add the documents to it

                print(f"Saving document embeddings of {foldername} under {vector_db_path}")
                db = Chroma.from_documents(collection_name=foldername, documents=text_chunks, embedding=embed_model, persist_directory=vector_db_path)
                db.persist()

In [7]:
#VC
#import chromadb
#chroma_client = chromadb.Client()
#chroma_client.list_collections()

In [8]:
#VC
#import chromadb
#from chromadb.config import Settings
# Example setup of the client to connect to your chroma server
#client = chromadb.HttpClient(host='localhost', port=8000)

In [8]:
def get_similar_chunks(question, similar_doc_count, colname):
    db_sem = Chroma(collection_name=colname, embedding_function=embed_model, persist_directory=vector_db_path)
    return db_sem.similarity_search(question, k=similar_doc_count)

In [10]:
#BRAVEN
#("I haven’t gotten any vaccines yet this year. Will I owe a copay if I get the flu shot?"):
#("What other vaccines can I get this year without owing a copay?", 3, "Braven"):
#("Will I owe a copay for lab services?", 3, "Braven"):
#("Will I owe a copay if I get an MRI?", 3, "Braven"):
#("My doctor recommended that I go to physical therapy.  Would that be covered? ", 3, "Braven"):
#("I am having surgery performed at a surgical center.  How much will I have to pay?", 3, "Braven"):
#("I am traveling out of the country and would like to know if I have coverage worldwide.", 3, "Braven"):
#("Would I owe a copayment for outpatient mental health services?", 3, "Braven"):
#("How do I find doctors that are in-network with my plan?", 3, "Braven"):
#("Can I go to an out-of-network provider?", 3, "Braven"):
#("Do I have to pay a deductible when using this medical plan?", 3, "Braven"):
#("I have to go for my annual physical exam.  How much will I have to pay for the visit?", 3, "Braven"):
#("Is there a nurse that I can speak with regarding a recent diagnosis I received?", 3, "Braven"):
#("Are hearing aids covered under my plan?", 3, "Braven"):
#("Do I have to pay for my annual dental cleaning?", 3, "Braven"):

In [9]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain.chains.question_answering import load_qa_chain
from langchain.memory import ConversationSummaryBufferMemory


def build_qa_chain():
    torch.cuda.empty_cache()
    model_name = "databricks/dolly-v2-2-8b" 

    instruct_pipeline = pipeline(model=model_name, torch_dtype=torch.bfloat16, trust_remote_code=True, device_map="auto", return_full_text=True, max_new_tokens=256, top_p=0.95, top_k=maxSemResCount)

    # Defining our prompt content.langchain will load our similar documents as {context}
    template = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

    Instructions:
    You are a customer service representative at a health insurance company and your job is to help providing the best accurate answer to a customer. 
    1. Limit your response to the information provided in this prompt. Do not conduct refer to external sources.
    2. Focus on the infromation rpesented here to form your answer.
    3. Do not include personal opinions or assumptions; stick to the facts provided in the prompt.
    4. Explain the answer with reference to the provided paragraphs. If the answer is not in the paragraphs, say that you do not know.
    5. Write your answer below the prompt.

    {context}

    Question: {question}

    Response:
    """
    prompt = PromptTemplate(input_variables=['context', 'question'], template=template)

    hf_pipe = HuggingFacePipeline(pipeline=instruct_pipeline)

    #summarizing the chat history
    #summarize_model = AutoModelForSeq2SeqLM.from_pretrained("t5-small", device_map="auto", torch_dtype=torch.bfloat16, trust_remote_code=True)
    #summarize_tokenizer = AutoTokenizer.from_pretrained("t5-small", padding_side="left", model_max_length = 512)
    #pipe_summary = pipeline("summarization", model=summarize_model, tokenizer=summarize_tokenizer) #, max_new_tokens=500, min_new_tokens=300
    # langchain pipeline doesn't support summarization yet, we added it as temp fix in the companion notebook _resources/00-init 
    #hf_summary = HuggingFacePipeline_WithSummarization(pipeline=pipe_summary)
    #will keep 500 token and then ask for a summary. Removes prefix as our model isn't trained on specific chat prefix and can get confused.
    #memory = ConversationSummaryBufferMemory(llm=hf_summary, memory_key="chat_history", input_key="question", max_token_limit=500, human_prefix = "", ai_prefix = "")

    return load_qa_chain(llm=hf_pipe, chain_type="stuff", prompt=prompt, verbose=True)

qa_chain = build_qa_chain()

A new version of the following files was downloaded from https://huggingface.co/databricks/dolly-v2-2-8b:
- instruct_pipeline.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


In [10]:
db_YA=Chroma(collection_name="CoverageofYoungAdultsinNJ", embedding_function=embed_model, persist_directory=vector_db_path)

for chunk in get_similar_chunks("I am switching employers, will this impact my child’s DU31 plan?", 3, "Braven"):
    print("***************************New Chunk:\n")
    print(chunk)

Using embedded DuckDB with persistence: data will be stored in: AIProjectDB/vector_db
Using embedded DuckDB with persistence: data will be stored in: AIProjectDB/vector_db


***************************New Chunk:

page_content='Summary of Benefits\n\nBraven Medicare Choice (PPO)\n\nJanuary 1, 2023 – December 31, 2023\n\nService area for this plan includes: Bergen, Essex, Hudson, Middlesex, Monmouth, Ocean, Passaic and Union counties.\n\nY0159_H0885_001_SB_2023_M\n\nThis booklet gives you a summary of what we cover and what you pay. It doesn’t list every service we cover or list every limitation or exclusion. To get a complete list of services, cost shares and exclusions, please refer to our Evidence of Coverage, which can be found online at BravenHealth.com/2023EOCChoice. Or, you can call us at 1-833-272-8360 (TTY 711) to request a mailed copy. Hours of operation are: October 1 – March 31: Monday – Sunday, from 8:00 a.m. to 8:00 p.m., ET and April 1 – September 30: Monday – Friday, from 8:00 a.m. to 8:00 p.m., ET.' metadata={'source': 'KnowledgeBaseDocs/Braven/BravenPPOBenefits (Bergen).pdf'}
***************************New Chunk:

page_content='Premiums and

In [11]:
def session():
    plan = input("What plan does the customer have? Enter BR for braven, MED for medicare, IC for insurance claim form,\n YA for Young Adult Coverage, AS for Assistant at Surgery, NS for Nutritional Support GEN for general, or ALL for everything.\n")
    if plan.upper()=="BR" or plan.upper()=="BRAVEN":
        #search from braven documentation
        return "Braven"
    elif plan.upper()=="MED" or plan.upper()=="MEDICARE":
        #search from medicare documentation
        return "Medicare"
    elif plan.upper()=="YA" or plan.upper()=="YOUNGADULT":
        return "CoverageofYoungAdultsinNJ"
    elif plan.upper()=="GEN" or plan.upper()=="GENERAL":
        #search general documents
        print("general") #NEEDS TO BE CHANGED
    elif plan.upper()=="IC" or plan.upper()=="INSURANCE CLAIM":
        return "HorizonMedicalHealthInsuranceClaimForm"
    elif plan.upper()=="AS" or plan.upper()=="ASSISTANT SURGERY":
        return "AssistantAtSurgery"
    elif plan.upper()=="NS" or plan.upper()=="NUTRITIONAL SUPPORT":
        return "NutrionalSupport"#"NutritionalSupport"
    elif plan.upper()=="AN" or plan.upper()=="ANESTHESIA":
        return "Anesthesia"
    elif plan.upper()=="ALL":
        #search all plans
        return "public_docs"
    else:
        print("Sorry, I'm not sure what you're saying")

In [12]:
def continueprompt():
    while True:
        ret = input("Would you like to ask a related question? Enter Y for yes and N for no.")
        if ret.upper()=="Y" or ret.upper()=="YES":
            return True
        elif ret.upper()=="N" or ret.upper()=="NO":
            return False
        else:
            print("I'm sorry, I didn't understand. Would you like to ask a related question? Enter Y for yes and N for no.")

In [13]:
def answer_question(question):
    plan = session()
    similar_chunks = get_similar_chunks(question, similar_doc_count=3, colname=plan)
    result = qa_chain({"input_documents": similar_chunks, "question": question})
    for d in result["input_documents"]:
        source_id = d.metadata["source"]
    print(source_id)
    res = question + result['output_text']
    print(res)
    if continueprompt()==True:
        newq = input("Please enter your next question.")
        answer_question(newq)

In [14]:
#BRAVEN
from langchain.vectorstores import Chroma
answer_question("I haven’t gotten any vaccines yet this year. Will I owe a copay if I get the flu shot?")

What plan does the customer have? Enter BR for braven, MED for medicare, IC for insurance claim form,
 YA for Young Adult Coverage, AS for Assistant at Surgery, NS for Nutritional Support GEN for general, or ALL for everything.
 br


Using embedded DuckDB with persistence: data will be stored in: AIProjectDB/vector_db




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

    Instructions:
    You are a customer service representative at a health insurance company and your job is to help providing the best accurate answer to a customer. 
    1. Limit your response to the information provided in this prompt. Do not conduct refer to external sources.
    2. Focus on the infromation rpesented here to form your answer.
    3. Do not include personal opinions or assumptions; stick to the facts provided in the prompt.
    4. Explain the answer with reference to the provided paragraphs. If the answer is not in the paragraphs, say that you do not know.
    5. Write your answer below the prompt.

    (counseling for people with no sign of tobacco-related disease)

Vaccines, including Pneumonia, Flu shots, Hepatitis B,

COVID-19 and other vaccines

“Welc

Would you like to ask a related question? Enter Y for yes and N for no. n


In [17]:
#BRAVEN
from langchain.vectorstores import Chroma
answer_question("I haven’t gotten any vaccines yet this year. Will I owe a copay if I get the flu shot?")

What plan does the customer have? Enter BR for braven, MED for medicare, IC for insurance claim form,
 YA for Young Adult Coverage, AS for Assistant at Surgery, NS for Nutritional Support GEN for general, or ALL for everything.
 br


Using embedded DuckDB with persistence: data will be stored in: AIProjectDB/vector_db




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

    Instructions:
    You are a customer service representative at a health insurance company and your job is to help providing the best accurate answer to a customer. 
    1. Limit your response to the information provided in this prompt. Do not conduct refer to external sources.
    2. Focus on the infromation rpesented here to form your answer.
    3. Do not include personal opinions or assumptions; stick to the facts provided in the prompt.
    4. Explain the answer with reference to the provided paragraphs. If the answer is not in the paragraphs, say that you do not know.
    5. Write your answer below the prompt.

    (counseling for people with no sign of tobacco-related disease)

Vaccines, including Pneumonia, Flu shots, Hepatitis B,

COVID-19 and other vaccines

“Welc

Would you like to ask a related question? Enter Y for yes and N for no. n


In [18]:
#BRAVEN
answer_question("What other vaccines have a $0 copay?")

What plan does the customer have? Enter BR for braven, MED for medicare, IC for insurance claim form,
 YA for Young Adult Coverage, AS for Assistant at Surgery, NS for Nutritional Support GEN for general, or ALL for everything.
 br


Using embedded DuckDB with persistence: data will be stored in: AIProjectDB/vector_db




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

    Instructions:
    You are a customer service representative at a health insurance company and your job is to help providing the best accurate answer to a customer. 
    1. Limit your response to the information provided in this prompt. Do not conduct refer to external sources.
    2. Focus on the infromation rpesented here to form your answer.
    3. Do not include personal opinions or assumptions; stick to the facts provided in the prompt.
    4. Explain the answer with reference to the provided paragraphs. If the answer is not in the paragraphs, say that you do not know.
    5. Write your answer below the prompt.

    (counseling for people with no sign of tobacco-related disease)

Vaccines, including Pneumonia, Flu shots, Hepatitis B,

COVID-19 and other vaccines

“Welc

Would you like to ask a related question? Enter Y for yes and N for no. n


In [19]:
#BRAVEN
answer_question("Will I owe a copay for lab services?")

What plan does the customer have? Enter BR for braven, MED for medicare, IC for insurance claim form,
 YA for Young Adult Coverage, AS for Assistant at Surgery, NS for Nutritional Support GEN for general, or ALL for everything.
 br


Using embedded DuckDB with persistence: data will be stored in: AIProjectDB/vector_db




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

    Instructions:
    You are a customer service representative at a health insurance company and your job is to help providing the best accurate answer to a customer. 
    1. Limit your response to the information provided in this prompt. Do not conduct refer to external sources.
    2. Focus on the infromation rpesented here to form your answer.
    3. Do not include personal opinions or assumptions; stick to the facts provided in the prompt.
    4. Explain the answer with reference to the provided paragraphs. If the answer is not in the paragraphs, say that you do not know.
    5. Write your answer below the prompt.

    Lab Services:



In-network:

o $0 copayment

Out-of-network:

o $20 copayment at an office o $50 copayment at an outpatient hospital

Covered Benefits NOT

Would you like to ask a related question? Enter Y for yes and N for no. n


In [20]:
answer_question("Who is the president of the US?")

What plan does the customer have? Enter BR for braven, MED for medicare, IC for insurance claim form,
 YA for Young Adult Coverage, AS for Assistant at Surgery, NS for Nutritional Support GEN for general, or ALL for everything.
 br


Using embedded DuckDB with persistence: data will be stored in: AIProjectDB/vector_db




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

    Instructions:
    You are a customer service representative at a health insurance company and your job is to help providing the best accurate answer to a customer. 
    1. Limit your response to the information provided in this prompt. Do not conduct refer to external sources.
    2. Focus on the infromation rpesented here to form your answer.
    3. Do not include personal opinions or assumptions; stick to the facts provided in the prompt.
    4. Explain the answer with reference to the provided paragraphs. If the answer is not in the paragraphs, say that you do not know.
    5. Write your answer below the prompt.

    Y0159_ECN008565SB2023_M

Notice of Nondiscrimination

Braven Health complies with applicable Federal civil rights laws and does not discriminate against no

Would you like to ask a related question? Enter Y for yes and N for no. n


In [21]:
#BRAVEN
answer_question("Will I owe a copay if I get an MRI?")

What plan does the customer have? Enter BR for braven, MED for medicare, IC for insurance claim form,
 YA for Young Adult Coverage, AS for Assistant at Surgery, NS for Nutritional Support GEN for general, or ALL for everything.
 br


Using embedded DuckDB with persistence: data will be stored in: AIProjectDB/vector_db




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

    Instructions:
    You are a customer service representative at a health insurance company and your job is to help providing the best accurate answer to a customer. 
    1. Limit your response to the information provided in this prompt. Do not conduct refer to external sources.
    2. Focus on the infromation rpesented here to form your answer.
    3. Do not include personal opinions or assumptions; stick to the facts provided in the prompt.
    4. Explain the answer with reference to the provided paragraphs. If the answer is not in the paragraphs, say that you do not know.
    5. Write your answer below the prompt.

    $40 copayment • $95 copayment for worldwide coverage

Urgently Needed Services

Copayment waived if admitted to a hospital within 24 hours for the same con

Would you like to ask a related question? Enter Y for yes and N for no. n


In [22]:
#BRAVEN
answer_question("My doctor recommended that I go to physical therapy.  Would that be covered?")

What plan does the customer have? Enter BR for braven, MED for medicare, IC for insurance claim form,
 YA for Young Adult Coverage, AS for Assistant at Surgery, NS for Nutritional Support GEN for general, or ALL for everything.
 br


Using embedded DuckDB with persistence: data will be stored in: AIProjectDB/vector_db




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

    Instructions:
    You are a customer service representative at a health insurance company and your job is to help providing the best accurate answer to a customer. 
    1. Limit your response to the information provided in this prompt. Do not conduct refer to external sources.
    2. Focus on the infromation rpesented here to form your answer.
    3. Do not include personal opinions or assumptions; stick to the facts provided in the prompt.
    4. Explain the answer with reference to the provided paragraphs. If the answer is not in the paragraphs, say that you do not know.
    5. Write your answer below the prompt.

    Physical Therapy1

Ambulance1

In-network:

Ground ambulance (one way): $250 copayment • Air ambulance (one way): $250 copayment

Out-of-network:

Emergenc

Would you like to ask a related question? Enter Y for yes and N for no. n


In [23]:
#BRAVEN
answer_question("I am having surgery performed at a surgical center.  How much will I have to pay?")

What plan does the customer have? Enter BR for braven, MED for medicare, IC for insurance claim form,
 YA for Young Adult Coverage, AS for Assistant at Surgery, NS for Nutritional Support GEN for general, or ALL for everything.
 br


Using embedded DuckDB with persistence: data will be stored in: AIProjectDB/vector_db




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

    Instructions:
    You are a customer service representative at a health insurance company and your job is to help providing the best accurate answer to a customer. 
    1. Limit your response to the information provided in this prompt. Do not conduct refer to external sources.
    2. Focus on the infromation rpesented here to form your answer.
    3. Do not include personal opinions or assumptions; stick to the facts provided in the prompt.
    4. Explain the answer with reference to the provided paragraphs. If the answer is not in the paragraphs, say that you do not know.
    5. Write your answer below the prompt.

    $40 copayment • $95 copayment for worldwide coverage

Urgently Needed Services

Copayment waived if admitted to a hospital within 24 hours for the same con

Would you like to ask a related question? Enter Y for yes and N for no. n


In [24]:
#BRAVEN
answer_question("I am traveling out of the country and would like to know if I have coverage worldwide.")

What plan does the customer have? Enter BR for braven, MED for medicare, IC for insurance claim form,
 YA for Young Adult Coverage, AS for Assistant at Surgery, NS for Nutritional Support GEN for general, or ALL for everything.
 br


Using embedded DuckDB with persistence: data will be stored in: AIProjectDB/vector_db




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

    Instructions:
    You are a customer service representative at a health insurance company and your job is to help providing the best accurate answer to a customer. 
    1. Limit your response to the information provided in this prompt. Do not conduct refer to external sources.
    2. Focus on the infromation rpesented here to form your answer.
    3. Do not include personal opinions or assumptions; stick to the facts provided in the prompt.
    4. Explain the answer with reference to the provided paragraphs. If the answer is not in the paragraphs, say that you do not know.
    5. Write your answer below the prompt.

    $40 copayment • $95 copayment for worldwide coverage

Urgently Needed Services

Copayment waived if admitted to a hospital within 24 hours for the same con

Would you like to ask a related question? Enter Y for yes and N for no. n


In [25]:
#BRAVEN
answer_question("Would I owe a copayment for outpatient mental health services?")

What plan does the customer have? Enter BR for braven, MED for medicare, IC for insurance claim form,
 YA for Young Adult Coverage, AS for Assistant at Surgery, NS for Nutritional Support GEN for general, or ALL for everything.
 br


Using embedded DuckDB with persistence: data will be stored in: AIProjectDB/vector_db




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

    Instructions:
    You are a customer service representative at a health insurance company and your job is to help providing the best accurate answer to a customer. 
    1. Limit your response to the information provided in this prompt. Do not conduct refer to external sources.
    2. Focus on the infromation rpesented here to form your answer.
    3. Do not include personal opinions or assumptions; stick to the facts provided in the prompt.
    4. Explain the answer with reference to the provided paragraphs. If the answer is not in the paragraphs, say that you do not know.
    5. Write your answer below the prompt.

    Our plan covers up to 190 days in a lifetime for inpatient

mental health care in a psychiatric hospital. Outpatient individual or group therapy office vis

Would you like to ask a related question? Enter Y for yes and N for no. n


In [26]:
#BRAVEN
answer_question("How do I find doctors that are in-network with my plan?")

What plan does the customer have? Enter BR for braven, MED for medicare, IC for insurance claim form,
 YA for Young Adult Coverage, AS for Assistant at Surgery, NS for Nutritional Support GEN for general, or ALL for everything.
 br


Using embedded DuckDB with persistence: data will be stored in: AIProjectDB/vector_db
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/transformers/pipelines/base.py:1080: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

    Instructions:
    You are a customer service representative at a health insurance company and your job is to help providing the best accurate answer to a customer. 
    1. Limit your response to the information provided in this prompt. Do not conduct refer to external sources.
    2. Focus on the infromation rpesented here to form your answer.
    3. Do not include personal opinions or assumptions; stick to the facts provided in the prompt.
    4. Explain the answer with reference to the provided paragraphs. If the answer is not in the paragraphs, say that you do not know.
    5. Write your answer below the prompt.

    Network providers and pharmacies

Braven Medicare Choice (PPO) has a network of doctors, hospitals, pharmacies and other providers. If you use the provider

Would you like to ask a related question? Enter Y for yes and N for no. n


In [27]:
#BRAVEN
answer_question("Can I go to an out-of-network provider?")

What plan does the customer have? Enter BR for braven, MED for medicare, IC for insurance claim form,
 YA for Young Adult Coverage, AS for Assistant at Surgery, NS for Nutritional Support GEN for general, or ALL for everything.
 br


Using embedded DuckDB with persistence: data will be stored in: AIProjectDB/vector_db




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

    Instructions:
    You are a customer service representative at a health insurance company and your job is to help providing the best accurate answer to a customer. 
    1. Limit your response to the information provided in this prompt. Do not conduct refer to external sources.
    2. Focus on the infromation rpesented here to form your answer.
    3. Do not include personal opinions or assumptions; stick to the facts provided in the prompt.
    4. Explain the answer with reference to the provided paragraphs. If the answer is not in the paragraphs, say that you do not know.
    5. Write your answer below the prompt.

    Out-of-network/non-contracted providers are under no obligation to treat plan members, except in emergency situations. Please call our customer service num

Would you like to ask a related question? Enter Y for yes and N for no. n


In [28]:
#BRAVEN
answer_question("Do I have to pay a deductible when using this medical plan?")

What plan does the customer have? Enter BR for braven, MED for medicare, IC for insurance claim form,
 YA for Young Adult Coverage, AS for Assistant at Surgery, NS for Nutritional Support GEN for general, or ALL for everything.
 br


Using embedded DuckDB with persistence: data will be stored in: AIProjectDB/vector_db




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

    Instructions:
    You are a customer service representative at a health insurance company and your job is to help providing the best accurate answer to a customer. 
    1. Limit your response to the information provided in this prompt. Do not conduct refer to external sources.
    2. Focus on the infromation rpesented here to form your answer.
    3. Do not include personal opinions or assumptions; stick to the facts provided in the prompt.
    4. Explain the answer with reference to the provided paragraphs. If the answer is not in the paragraphs, say that you do not know.
    5. Write your answer below the prompt.

    Important Message About What You Pay for Vaccines – Our plan covers most Part D vaccines at no cost to you even if you haven’t paid your deductible. Call M

Would you like to ask a related question? Enter Y for yes and N for no. n


In [29]:
#BRAVEN
answer_question("I have to go for my annual physical exam.  How much will I have to pay for the visit?")

What plan does the customer have? Enter BR for braven, MED for medicare, IC for insurance claim form,
 YA for Young Adult Coverage, AS for Assistant at Surgery, NS for Nutritional Support GEN for general, or ALL for everything.
 br


Using embedded DuckDB with persistence: data will be stored in: AIProjectDB/vector_db




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

    Instructions:
    You are a customer service representative at a health insurance company and your job is to help providing the best accurate answer to a customer. 
    1. Limit your response to the information provided in this prompt. Do not conduct refer to external sources.
    2. Focus on the infromation rpesented here to form your answer.
    3. Do not include personal opinions or assumptions; stick to the facts provided in the prompt.
    4. Explain the answer with reference to the provided paragraphs. If the answer is not in the paragraphs, say that you do not know.
    5. Write your answer below the prompt.

    $40 copayment • $95 copayment for worldwide coverage

Urgently Needed Services

Copayment waived if admitted to a hospital within 24 hours for the same con

Would you like to ask a related question? Enter Y for yes and N for no. n


In [30]:
#BRAVEN
answer_question("Is there a nurse that I can speak with regarding a recent diagnosis I received?")

What plan does the customer have? Enter BR for braven, MED for medicare, IC for insurance claim form,
 YA for Young Adult Coverage, AS for Assistant at Surgery, NS for Nutritional Support GEN for general, or ALL for everything.
 br


Using embedded DuckDB with persistence: data will be stored in: AIProjectDB/vector_db




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

    Instructions:
    You are a customer service representative at a health insurance company and your job is to help providing the best accurate answer to a customer. 
    1. Limit your response to the information provided in this prompt. Do not conduct refer to external sources.
    2. Focus on the infromation rpesented here to form your answer.
    3. Do not include personal opinions or assumptions; stick to the facts provided in the prompt.
    4. Explain the answer with reference to the provided paragraphs. If the answer is not in the paragraphs, say that you do not know.
    5. Write your answer below the prompt.

    French: Nous proposons des services gratuits d'interprétation pour répondre à toutes vos questions relatives à notre régime de santé ou d'assurance-médicam

Would you like to ask a related question? Enter Y for yes and N for no. n


In [31]:
#BRAVEN
answer_question("Are hearing aids covered under my plan?")

What plan does the customer have? Enter BR for braven, MED for medicare, IC for insurance claim form,
 YA for Young Adult Coverage, AS for Assistant at Surgery, NS for Nutritional Support GEN for general, or ALL for everything.
 br


Using embedded DuckDB with persistence: data will be stored in: AIProjectDB/vector_db




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

    Instructions:
    You are a customer service representative at a health insurance company and your job is to help providing the best accurate answer to a customer. 
    1. Limit your response to the information provided in this prompt. Do not conduct refer to external sources.
    2. Focus on the infromation rpesented here to form your answer.
    3. Do not include personal opinions or assumptions; stick to the facts provided in the prompt.
    4. Explain the answer with reference to the provided paragraphs. If the answer is not in the paragraphs, say that you do not know.
    5. Write your answer below the prompt.

    Fitting/Evaluation for hearing aid (1 per year):

In-network: $0 copayment • Out-of-network: $30 copayment • Call HearUSA to schedule a visit with an in-ne

Would you like to ask a related question? Enter Y for yes and N for no. n


In [32]:
#BRAVEN
answer_question("Do I have to pay for my annual dental cleaning?")

What plan does the customer have? Enter BR for braven, MED for medicare, IC for insurance claim form,
 YA for Young Adult Coverage, AS for Assistant at Surgery, NS for Nutritional Support GEN for general, or ALL for everything.
 br


Using embedded DuckDB with persistence: data will be stored in: AIProjectDB/vector_db




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

    Instructions:
    You are a customer service representative at a health insurance company and your job is to help providing the best accurate answer to a customer. 
    1. Limit your response to the information provided in this prompt. Do not conduct refer to external sources.
    2. Focus on the infromation rpesented here to form your answer.
    3. Do not include personal opinions or assumptions; stick to the facts provided in the prompt.
    4. Explain the answer with reference to the provided paragraphs. If the answer is not in the paragraphs, say that you do not know.
    5. Write your answer below the prompt.

    Dental Services



In- and Out-of-network:

o $0 copayment for cleaning (up to 3 per year) o $0 copayment for fluoride treatment (1 every 6

months)

o $0 

Would you like to ask a related question? Enter Y for yes and N for no. n


In [33]:
#Horizon Med Claim Form
answer_question("Can you direct me to the form used to submit a health claim for reimbursement?")

What plan does the customer have? Enter BR for braven, MED for medicare, IC for insurance claim form,
 YA for Young Adult Coverage, AS for Assistant at Surgery, NS for Nutritional Support GEN for general, or ALL for everything.
 br


Using embedded DuckDB with persistence: data will be stored in: AIProjectDB/vector_db




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

    Instructions:
    You are a customer service representative at a health insurance company and your job is to help providing the best accurate answer to a customer. 
    1. Limit your response to the information provided in this prompt. Do not conduct refer to external sources.
    2. Focus on the infromation rpesented here to form your answer.
    3. Do not include personal opinions or assumptions; stick to the facts provided in the prompt.
    4. Explain the answer with reference to the provided paragraphs. If the answer is not in the paragraphs, say that you do not know.
    5. Write your answer below the prompt.

    OCR Complaint forms are available at www.hhs.gov/ocr/office/file/index.html.

Para ayuda en español, llame a 1-844-498-9393 (TTY 711).

Y0159_ECN007353B_C


Would you like to ask a related question? Enter Y for yes and N for no. n


In [34]:
#Horizon Med Claim Form
answer_question("What is the address that I can mail this to?")

What plan does the customer have? Enter BR for braven, MED for medicare, IC for insurance claim form,
 YA for Young Adult Coverage, AS for Assistant at Surgery, NS for Nutritional Support GEN for general, or ALL for everything.
 br


Using embedded DuckDB with persistence: data will be stored in: AIProjectDB/vector_db




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

    Instructions:
    You are a customer service representative at a health insurance company and your job is to help providing the best accurate answer to a customer. 
    1. Limit your response to the information provided in this prompt. Do not conduct refer to external sources.
    2. Focus on the infromation rpesented here to form your answer.
    3. Do not include personal opinions or assumptions; stick to the facts provided in the prompt.
    4. Explain the answer with reference to the provided paragraphs. If the answer is not in the paragraphs, say that you do not know.
    5. Write your answer below the prompt.

    Summary of Benefits

Braven Medicare Choice (PPO)

January 1, 2023 – December 31, 2023

Service area for this plan includes: Bergen, Essex, Hudson, Middles

Would you like to ask a related question? Enter Y for yes and N for no. u


I'm sorry, I didn't understand. Would you like to ask a related question? Enter Y for yes and N for no.


Would you like to ask a related question? Enter Y for yes and N for no. n


In [ ]:
answer_question("What is the criteria to buy a Horizon Dental insurance plan?")

In [ ]:
answer_question("What is a phone number for dental plans?")

In [ ]:
answer_question("Is there a dental plan for kids?")

In [ ]:
answer_question("Will I owe a copay if I get an MRI?")

In [ ]:
answer_question("Will I owe a copay for lab services?")

In [ ]:
answer_question("My doctor recommended physical therapy. Would it be covered?")

In [ ]:
answer_question("I am travelling out of the country and would like to know if I have worldwide coverage with Braven.")

In [ ]:
answer_question("Does my plan cover gym memberships or fitness classes?")

In [ ]:
answer_question("Does Medicare cover religious care?")

In [ ]:
answer_question("Will I owe a copay for lab services?") #Braven

In [ ]:
answer_question("Would I owe a copayment for outpatient mental health services?")

In [36]:
#YA Coverage
answer_question("My son is 28 years old. Can I add him to my job’s health insurance policy by taking the New Jersey Dependent Under 31 (DU31) election?")

What plan does the customer have? Enter BR for braven, MED for medicare, IC for insurance claim form,
 YA for Young Adult Coverage, AS for Assistant at Surgery, NS for Nutritional Support GEN for general, or ALL for everything.
 ya


Using embedded DuckDB with persistence: data will be stored in: AIProjectDB/vector_db




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

    Instructions:
    You are a customer service representative at a health insurance company and your job is to help providing the best accurate answer to a customer. 
    1. Limit your response to the information provided in this prompt. Do not conduct refer to external sources.
    2. Focus on the infromation rpesented here to form your answer.
    3. Do not include personal opinions or assumptions; stick to the facts provided in the prompt.
    4. Explain the answer with reference to the provided paragraphs. If the answer is not in the paragraphs, say that you do not know.
    5. Write your answer below the prompt.

    The Dependent Under 31 Election (DU31)

https://www.state.nj.us/dobi/division_consumers/du31.html#whateligibility

1/15

8/7/23, 2:39 PM

Coverage of Young

Would you like to ask a related question? Enter Y for yes and N for no. n


In [37]:
#YA Coverage
answer_question("My group health plan is not subject to the DU31 law.  Are there any other options for my child?")

What plan does the customer have? Enter BR for braven, MED for medicare, IC for insurance claim form,
 YA for Young Adult Coverage, AS for Assistant at Surgery, NS for Nutritional Support GEN for general, or ALL for everything.
 ya


Using embedded DuckDB with persistence: data will be stored in: AIProjectDB/vector_db




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

    Instructions:
    You are a customer service representative at a health insurance company and your job is to help providing the best accurate answer to a customer. 
    1. Limit your response to the information provided in this prompt. Do not conduct refer to external sources.
    2. Focus on the infromation rpesented here to form your answer.
    3. Do not include personal opinions or assumptions; stick to the facts provided in the prompt.
    4. Explain the answer with reference to the provided paragraphs. If the answer is not in the paragraphs, say that you do not know.
    5. Write your answer below the prompt.

    Other possible options and alternatives:

If a young adult is working, he or she may have the option to enroll in his or her employer’s own group health pl

Would you like to ask a related question? Enter Y for yes and N for no. n


In [38]:
#YA Coverage
answer_question("I am turning 27 and am currently on my parent’s plan through their employer.  Will I be able to continue my coverage?")

What plan does the customer have? Enter BR for braven, MED for medicare, IC for insurance claim form,
 YA for Young Adult Coverage, AS for Assistant at Surgery, NS for Nutritional Support GEN for general, or ALL for everything.
 ya


Using embedded DuckDB with persistence: data will be stored in: AIProjectDB/vector_db




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

    Instructions:
    You are a customer service representative at a health insurance company and your job is to help providing the best accurate answer to a customer. 
    1. Limit your response to the information provided in this prompt. Do not conduct refer to external sources.
    2. Focus on the infromation rpesented here to form your answer.
    3. Do not include personal opinions or assumptions; stick to the facts provided in the prompt.
    4. Explain the answer with reference to the provided paragraphs. If the answer is not in the paragraphs, say that you do not know.
    5. Write your answer below the prompt.

    parent’s household, or a full-time student. The young adult may remain covered under his or her parent’s plan even if he or she marries. (However, neither 

Would you like to ask a related question? Enter Y for yes and N for no. n


In [39]:
#YA Coverage
answer_question("How can I enroll my daughter in a DU31 plan?")

What plan does the customer have? Enter BR for braven, MED for medicare, IC for insurance claim form,
 YA for Young Adult Coverage, AS for Assistant at Surgery, NS for Nutritional Support GEN for general, or ALL for everything.
 ya


Using embedded DuckDB with persistence: data will be stored in: AIProjectDB/vector_db




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

    Instructions:
    You are a customer service representative at a health insurance company and your job is to help providing the best accurate answer to a customer. 
    1. Limit your response to the information provided in this prompt. Do not conduct refer to external sources.
    2. Focus on the infromation rpesented here to form your answer.
    3. Do not include personal opinions or assumptions; stick to the facts provided in the prompt.
    4. Explain the answer with reference to the provided paragraphs. If the answer is not in the paragraphs, say that you do not know.
    5. Write your answer below the prompt.

    When May I Enroll?

Generally, the enrollment period for DU31 eligibility is limited to the first 30 days following establishment of eligibility, or during

Would you like to ask a related question? Enter Y for yes and N for no. n


In [40]:
#YA Coverage
answer_question("Is there a cost associated with adding my child by taking the New Jersey Under 31 (DU31) election?")

What plan does the customer have? Enter BR for braven, MED for medicare, IC for insurance claim form,
 YA for Young Adult Coverage, AS for Assistant at Surgery, NS for Nutritional Support GEN for general, or ALL for everything.
 ya


Using embedded DuckDB with persistence: data will be stored in: AIProjectDB/vector_db




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

    Instructions:
    You are a customer service representative at a health insurance company and your job is to help providing the best accurate answer to a customer. 
    1. Limit your response to the information provided in this prompt. Do not conduct refer to external sources.
    2. Focus on the infromation rpesented here to form your answer.
    3. Do not include personal opinions or assumptions; stick to the facts provided in the prompt.
    4. Explain the answer with reference to the provided paragraphs. If the answer is not in the paragraphs, say that you do not know.
    5. Write your answer below the prompt.

    What are the Eligibility Requirements?

In order for a young adult to be eligible for a DU31 election option:

the group coverage must meet certain require

Would you like to ask a related question? Enter Y for yes and N for no. n


In [41]:
#YA Coverage
answer_question("I am switching employers, will this impact my child’s DU31 plan?")

What plan does the customer have? Enter BR for braven, MED for medicare, IC for insurance claim form,
 YA for Young Adult Coverage, AS for Assistant at Surgery, NS for Nutritional Support GEN for general, or ALL for everything.
 ya


Using embedded DuckDB with persistence: data will be stored in: AIProjectDB/vector_db




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

    Instructions:
    You are a customer service representative at a health insurance company and your job is to help providing the best accurate answer to a customer. 
    1. Limit your response to the information provided in this prompt. Do not conduct refer to external sources.
    2. Focus on the infromation rpesented here to form your answer.
    3. Do not include personal opinions or assumptions; stick to the facts provided in the prompt.
    4. Explain the answer with reference to the provided paragraphs. If the answer is not in the paragraphs, say that you do not know.
    5. Write your answer below the prompt.

    Self-funded plans of public employers are subject to the laws of the states where located. So, although the SHBP is self-funded, because P.L. 2005, C. 375 

Would you like to ask a related question? Enter Y for yes and N for no. n


In [42]:
#YA Coverage
answer_question("Can I enroll my son under a DU31 plan if he does not live in the same house as me?")

What plan does the customer have? Enter BR for braven, MED for medicare, IC for insurance claim form,
 YA for Young Adult Coverage, AS for Assistant at Surgery, NS for Nutritional Support GEN for general, or ALL for everything.
 ya


Using embedded DuckDB with persistence: data will be stored in: AIProjectDB/vector_db




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

    Instructions:
    You are a customer service representative at a health insurance company and your job is to help providing the best accurate answer to a customer. 
    1. Limit your response to the information provided in this prompt. Do not conduct refer to external sources.
    2. Focus on the infromation rpesented here to form your answer.
    3. Do not include personal opinions or assumptions; stick to the facts provided in the prompt.
    4. Explain the answer with reference to the provided paragraphs. If the answer is not in the paragraphs, say that you do not know.
    5. Write your answer below the prompt.

    10. So, to satisfy the second part of the DU31 eligibility test, must the parent have elected

dependent coverage before a young adult can make a DU31 elec

Would you like to ask a related question? Enter Y for yes and N for no. n


In [43]:
#Assistant At Surgery
answer_question("When was the Assistant at Surgery document's last revision date?")

What plan does the customer have? Enter BR for braven, MED for medicare, IC for insurance claim form,
 YA for Young Adult Coverage, AS for Assistant at Surgery, NS for Nutritional Support GEN for general, or ALL for everything.
 as


Using embedded DuckDB with persistence: data will be stored in: AIProjectDB/vector_db




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

    Instructions:
    You are a customer service representative at a health insurance company and your job is to help providing the best accurate answer to a customer. 
    1. Limit your response to the information provided in this prompt. Do not conduct refer to external sources.
    2. Focus on the infromation rpesented here to form your answer.
    3. Do not include personal opinions or assumptions; stick to the facts provided in the prompt.
    4. Explain the answer with reference to the provided paragraphs. If the answer is not in the paragraphs, say that you do not know.
    5. Write your answer below the prompt.

    8/7/23, 9:27 PM

Assistant at Surgery - Horizon Blue Cross Blue Shield of New Jersey

COVID-19

Stay informed. Get the latest information on COVID-19. (/pr

Would you like to ask a related question? Enter Y for yes and N for no. n


In [44]:
#Assistant At Surgery
answer_question("What is the scope?")

What plan does the customer have? Enter BR for braven, MED for medicare, IC for insurance claim form,
 YA for Young Adult Coverage, AS for Assistant at Surgery, NS for Nutritional Support GEN for general, or ALL for everything.
 as


Using embedded DuckDB with persistence: data will be stored in: AIProjectDB/vector_db




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

    Instructions:
    You are a customer service representative at a health insurance company and your job is to help providing the best accurate answer to a customer. 
    1. Limit your response to the information provided in this prompt. Do not conduct refer to external sources.
    2. Focus on the infromation rpesented here to form your answer.
    3. Do not include personal opinions or assumptions; stick to the facts provided in the prompt.
    4. Explain the answer with reference to the provided paragraphs. If the answer is not in the paragraphs, say that you do not know.
    5. Write your answer below the prompt.

    Scope:

All products are included, except products where Horizon BCBSNJ is secondary to Medicare (e.g. Medigap).

All Insured and Administrative Services O

Would you like to ask a related question? Enter Y for yes and N for no. n


In [57]:
#Assistant At Surgery
answer_question("What is a Modifier 82?")

What plan does the customer have? Enter BR for braven, MED for medicare, IC for insurance claim form,
 YA for Young Adult Coverage, AS for Assistant at Surgery, NS for Nutritional Support GEN for general, or ALL for everything.
 as


Using embedded DuckDB with persistence: data will be stored in: AIProjectDB/vector_db




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

    Instructions:
    You are a customer service representative at a health insurance company and your job is to help providing the best accurate answer to a customer. 
    1. Limit your response to the information provided in this prompt. Do not conduct refer to external sources.
    2. Focus on the infromation rpesented here to form your answer.
    3. Do not include personal opinions or assumptions; stick to the facts provided in the prompt.
    4. Explain the answer with reference to the provided paragraphs. If the answer is not in the paragraphs, say that you do not know.
    5. Write your answer below the prompt.

    Modi er 80: Used when the assistant surgeon services were provided by a medical doctor. Modi er 81: Used to identify minimum assistant surgeon services; on

Would you like to ask a related question? Enter Y for yes and N for no. n


In [45]:
#Assistant At Surgery
answer_question("What does a modifier 81 reimburse at?")

What plan does the customer have? Enter BR for braven, MED for medicare, IC for insurance claim form,
 YA for Young Adult Coverage, AS for Assistant at Surgery, NS for Nutritional Support GEN for general, or ALL for everything.
 as


Using embedded DuckDB with persistence: data will be stored in: AIProjectDB/vector_db




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

    Instructions:
    You are a customer service representative at a health insurance company and your job is to help providing the best accurate answer to a customer. 
    1. Limit your response to the information provided in this prompt. Do not conduct refer to external sources.
    2. Focus on the infromation rpesented here to form your answer.
    3. Do not include personal opinions or assumptions; stick to the facts provided in the prompt.
    4. Explain the answer with reference to the provided paragraphs. If the answer is not in the paragraphs, say that you do not know.
    5. Write your answer below the prompt.

    follows:

Modi er

Reimbursement

80

16% of applicable Horizon BCBSNJ fee schedule

81

16% of applicable Horizon BCBSNJ fee schedule

82

16% of applicab

Would you like to ask a related question? Enter Y for yes and N for no. n


In [46]:
#Assistant At Surgery
answer_question("Are there any limitations and exclusions?")

What plan does the customer have? Enter BR for braven, MED for medicare, IC for insurance claim form,
 YA for Young Adult Coverage, AS for Assistant at Surgery, NS for Nutritional Support GEN for general, or ALL for everything.
 as


Using embedded DuckDB with persistence: data will be stored in: AIProjectDB/vector_db




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

    Instructions:
    You are a customer service representative at a health insurance company and your job is to help providing the best accurate answer to a customer. 
    1. Limit your response to the information provided in this prompt. Do not conduct refer to external sources.
    2. Focus on the infromation rpesented here to form your answer.
    3. Do not include personal opinions or assumptions; stick to the facts provided in the prompt.
    4. Explain the answer with reference to the provided paragraphs. If the answer is not in the paragraphs, say that you do not know.
    5. Write your answer below the prompt.

    1/5

8/7/23, 9:27 PM

Assistant at Surgery - Horizon Blue Cross Blue Shield of New Jersey

In instances where the provider is not participating, member lia

Would you like to ask a related question? Enter Y for yes and N for no. n


In [52]:
#Nutritional Support
answer_question("When was the Nutritional Support policy's last review date?")

What plan does the customer have? Enter BR for braven, MED for medicare, IC for insurance claim form,
 YA for Young Adult Coverage, AS for Assistant at Surgery, NS for Nutritional Support GEN for general, or ALL for everything.
 ns


Using embedded DuckDB with persistence: data will be stored in: AIProjectDB/vector_db




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

    Instructions:
    You are a customer service representative at a health insurance company and your job is to help providing the best accurate answer to a customer. 
    1. Limit your response to the information provided in this prompt. Do not conduct refer to external sources.
    2. Focus on the infromation rpesented here to form your answer.
    3. Do not include personal opinions or assumptions; stick to the facts provided in the prompt.
    4. Explain the answer with reference to the provided paragraphs. If the answer is not in the paragraphs, say that you do not know.
    5. Write your answer below the prompt.

    CPT*

HCPCS

B4102

B4103

B4104

B4105

B4149

B4150

B4152

B4153

B4154

B4155

B4157

B4158

B4159

B4160

B4161

B4162

B4164

B4168

B4172

B4176

B4

Would you like to ask a related question? Enter Y for yes and N for no. n


In [53]:
#Nutritional Support
answer_question("What is the infant formula mandate?")

What plan does the customer have? Enter BR for braven, MED for medicare, IC for insurance claim form,
 YA for Young Adult Coverage, AS for Assistant at Surgery, NS for Nutritional Support GEN for general, or ALL for everything.
 ns


Using embedded DuckDB with persistence: data will be stored in: AIProjectDB/vector_db




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

    Instructions:
    You are a customer service representative at a health insurance company and your job is to help providing the best accurate answer to a customer. 
    1. Limit your response to the information provided in this prompt. Do not conduct refer to external sources.
    2. Focus on the infromation rpesented here to form your answer.
    3. Do not include personal opinions or assumptions; stick to the facts provided in the prompt.
    4. Explain the answer with reference to the provided paragraphs. If the answer is not in the paragraphs, say that you do not know.
    5. Write your answer below the prompt.

    Nutritional Support (Infant Formulas, Enteral Nutrition and Total Parenteral Nutrition) Print Policy | Questions? Email Us (mailto:medpol_ask@horizonblue.c

Would you like to ask a related question? Enter Y for yes and N for no. n


In [54]:
#Nutritional Support
answer_question("Is Enteral Nutrition medically necessary?")

What plan does the customer have? Enter BR for braven, MED for medicare, IC for insurance claim form,
 YA for Young Adult Coverage, AS for Assistant at Surgery, NS for Nutritional Support GEN for general, or ALL for everything.
 ns


Using embedded DuckDB with persistence: data will be stored in: AIProjectDB/vector_db




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

    Instructions:
    You are a customer service representative at a health insurance company and your job is to help providing the best accurate answer to a customer. 
    1. Limit your response to the information provided in this prompt. Do not conduct refer to external sources.
    2. Focus on the infromation rpesented here to form your answer.
    3. Do not include personal opinions or assumptions; stick to the facts provided in the prompt.
    4. Explain the answer with reference to the provided paragraphs. If the answer is not in the paragraphs, say that you do not know.
    5. Write your answer below the prompt.

    (Refer to policy statement III below)

III. Enteral Nutrition (EN):

A. Enteral nutrition is medically necessary when both of the following numbered criter

Would you like to ask a related question? Enter Y for yes and N for no. n


In [ ]:
#Nutritional Support
answer_question("When is Relizorb medically necessary?")

In [55]:
#Nutritional Support
answer_question("What are the main types of infant formulas?")

What plan does the customer have? Enter BR for braven, MED for medicare, IC for insurance claim form,
 YA for Young Adult Coverage, AS for Assistant at Surgery, NS for Nutritional Support GEN for general, or ALL for everything.
 ns


Using embedded DuckDB with persistence: data will be stored in: AIProjectDB/vector_db




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

    Instructions:
    You are a customer service representative at a health insurance company and your job is to help providing the best accurate answer to a customer. 
    1. Limit your response to the information provided in this prompt. Do not conduct refer to external sources.
    2. Focus on the infromation rpesented here to form your answer.
    3. Do not include personal opinions or assumptions; stick to the facts provided in the prompt.
    4. Explain the answer with reference to the provided paragraphs. If the answer is not in the paragraphs, say that you do not know.
    5. Write your answer below the prompt.

    life may not be feasible for all mothers. Infant formulas offer another option.

There are three (3) main types of infant formulas:

Cow’s milk formulas - 

Would you like to ask a related question? Enter Y for yes and N for no. n


In [56]:
#Nutritional Support
answer_question("What are the associated formula codes?")

What plan does the customer have? Enter BR for braven, MED for medicare, IC for insurance claim form,
 YA for Young Adult Coverage, AS for Assistant at Surgery, NS for Nutritional Support GEN for general, or ALL for everything.
 ns


Using embedded DuckDB with persistence: data will be stored in: AIProjectDB/vector_db




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

    Instructions:
    You are a customer service representative at a health insurance company and your job is to help providing the best accurate answer to a customer. 
    1. Limit your response to the information provided in this prompt. Do not conduct refer to external sources.
    2. Focus on the infromation rpesented here to form your answer.
    3. Do not include personal opinions or assumptions; stick to the facts provided in the prompt.
    4. Explain the answer with reference to the provided paragraphs. If the answer is not in the paragraphs, say that you do not know.
    5. Write your answer below the prompt.

    Codes:

(The list of codes is not intended to be all-inclusive and is included below for informational purposes only. Inclusion or exclusion of a procedure

Would you like to ask a related question? Enter Y for yes and N for no. n


In [ ]:
#Nutritional Support
#When was this policy last reviewed?
#What is the infant formula mandate?
#Is Enteral Nutrition medically necessary?
#What are the main types of infant formulas?
#What are the associated formula codes?

In [ ]:
import torch
from transformers import pipeline

generate_text=pipeline(model="databricks/dolly-v2-2-8b",
                       torch_dtype=torch.bfloat16,
                       trust_remote_code=True,
                       device_map="auto")

In [ ]:
#Anesthesia
#What is the definition of anesthesia?
#What is the definition of anesthesia time?
#What is the reimbursement formula?
#What is the desciption of Modifier AA?
#What is the reimbursement of Modifier QX?
#What modifier are informational only?
#Are their any limitations and exclusions?

In [15]:
#Anesthesia
answer_question("What is the definition of anesthesia?")

What plan does the customer have? Enter BR for braven, MED for medicare, IC for insurance claim form,
 YA for Young Adult Coverage, AS for Assistant at Surgery, NS for Nutritional Support GEN for general, or ALL for everything.
 an


Using embedded DuckDB with persistence: data will be stored in: AIProjectDB/vector_db




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

    Instructions:
    You are a customer service representative at a health insurance company and your job is to help providing the best accurate answer to a customer. 
    1. Limit your response to the information provided in this prompt. Do not conduct refer to external sources.
    2. Focus on the infromation rpesented here to form your answer.
    3. Do not include personal opinions or assumptions; stick to the facts provided in the prompt.
    4. Explain the answer with reference to the provided paragraphs. If the answer is not in the paragraphs, say that you do not know.
    5. Write your answer below the prompt.

    De nitions:

Anesthesia - the administration of a drug or anesthetic agent by an anesthesiologist in order for a patient to obtain muscular relaxation and 

Would you like to ask a related question? Enter Y for yes and N for no. n


In [16]:
#Anesthesia
answer_question("What is the definition of anesthesia time?")

What plan does the customer have? Enter BR for braven, MED for medicare, IC for insurance claim form,
 YA for Young Adult Coverage, AS for Assistant at Surgery, NS for Nutritional Support GEN for general, or ALL for everything.
 an


Using embedded DuckDB with persistence: data will be stored in: AIProjectDB/vector_db




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

    Instructions:
    You are a customer service representative at a health insurance company and your job is to help providing the best accurate answer to a customer. 
    1. Limit your response to the information provided in this prompt. Do not conduct refer to external sources.
    2. Focus on the infromation rpesented here to form your answer.
    3. Do not include personal opinions or assumptions; stick to the facts provided in the prompt.
    4. Explain the answer with reference to the provided paragraphs. If the answer is not in the paragraphs, say that you do not know.
    5. Write your answer below the prompt.

    De nitions:

Anesthesia - the administration of a drug or anesthetic agent by an anesthesiologist in order for a patient to obtain muscular relaxation and 

Would you like to ask a related question? Enter Y for yes and N for no. n


In [17]:
#Anesthesia
answer_question("What is the reimbursement formula?")

What plan does the customer have? Enter BR for braven, MED for medicare, IC for insurance claim form,
 YA for Young Adult Coverage, AS for Assistant at Surgery, NS for Nutritional Support GEN for general, or ALL for everything.
 an


Using embedded DuckDB with persistence: data will be stored in: AIProjectDB/vector_db




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

    Instructions:
    You are a customer service representative at a health insurance company and your job is to help providing the best accurate answer to a customer. 
    1. Limit your response to the information provided in this prompt. Do not conduct refer to external sources.
    2. Focus on the infromation rpesented here to form your answer.
    3. Do not include personal opinions or assumptions; stick to the facts provided in the prompt.
    4. Explain the answer with reference to the provided paragraphs. If the answer is not in the paragraphs, say that you do not know.
    5. Write your answer below the prompt.

    Modi er Description

Reimbursement

AA

Anesthesia Services performed personally by the anesthesiologist.

Services are paid at 100% of the applicable Hori

Would you like to ask a related question? Enter Y for yes and N for no. n


In [18]:
#Anesthesia
answer_question("What is the desciption of Modifier AA?")

What plan does the customer have? Enter BR for braven, MED for medicare, IC for insurance claim form,
 YA for Young Adult Coverage, AS for Assistant at Surgery, NS for Nutritional Support GEN for general, or ALL for everything.
 an


Using embedded DuckDB with persistence: data will be stored in: AIProjectDB/vector_db




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

    Instructions:
    You are a customer service representative at a health insurance company and your job is to help providing the best accurate answer to a customer. 
    1. Limit your response to the information provided in this prompt. Do not conduct refer to external sources.
    2. Focus on the infromation rpesented here to form your answer.
    3. Do not include personal opinions or assumptions; stick to the facts provided in the prompt.
    4. Explain the answer with reference to the provided paragraphs. If the answer is not in the paragraphs, say that you do not know.
    5. Write your answer below the prompt.

    **ITS home claims, Medicare Secondary, Federal Employee Health Program and claims submitted for the Horizon Medicare Advantage (MRSK) members when billed w

Would you like to ask a related question? Enter Y for yes and N for no. n


In [19]:
#Anesthesia
answer_question("What is the reimbursement of Modifier QX?")

What plan does the customer have? Enter BR for braven, MED for medicare, IC for insurance claim form,
 YA for Young Adult Coverage, AS for Assistant at Surgery, NS for Nutritional Support GEN for general, or ALL for everything.
 an


Using embedded DuckDB with persistence: data will be stored in: AIProjectDB/vector_db




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

    Instructions:
    You are a customer service representative at a health insurance company and your job is to help providing the best accurate answer to a customer. 
    1. Limit your response to the information provided in this prompt. Do not conduct refer to external sources.
    2. Focus on the infromation rpesented here to form your answer.
    3. Do not include personal opinions or assumptions; stick to the facts provided in the prompt.
    4. Explain the answer with reference to the provided paragraphs. If the answer is not in the paragraphs, say that you do not know.
    5. Write your answer below the prompt.

    **ITS home claims, Medicare Secondary, Federal Employee Health Program and claims submitted for the Horizon Medicare Advantage (MRSK) members when billed w

Would you like to ask a related question? Enter Y for yes and N for no. n


In [20]:
#Anesthesia
answer_question("What modifier are informational only?")

What plan does the customer have? Enter BR for braven, MED for medicare, IC for insurance claim form,
 YA for Young Adult Coverage, AS for Assistant at Surgery, NS for Nutritional Support GEN for general, or ALL for everything.
 an


Using embedded DuckDB with persistence: data will be stored in: AIProjectDB/vector_db




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

    Instructions:
    You are a customer service representative at a health insurance company and your job is to help providing the best accurate answer to a customer. 
    1. Limit your response to the information provided in this prompt. Do not conduct refer to external sources.
    2. Focus on the infromation rpesented here to form your answer.
    3. Do not include personal opinions or assumptions; stick to the facts provided in the prompt.
    4. Explain the answer with reference to the provided paragraphs. If the answer is not in the paragraphs, say that you do not know.
    5. Write your answer below the prompt.

    together, reimbursement will be at the lesser reimbursement percentage

Modi ers QS, G8 and G9 are informational only, and do not affect reimbursement. Inf

Would you like to ask a related question? Enter Y for yes and N for no. n


In [21]:
#Anesthesia
answer_question("Are their any limitations and exclusions?")

What plan does the customer have? Enter BR for braven, MED for medicare, IC for insurance claim form,
 YA for Young Adult Coverage, AS for Assistant at Surgery, NS for Nutritional Support GEN for general, or ALL for everything.
 an


Using embedded DuckDB with persistence: data will be stored in: AIProjectDB/vector_db




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

    Instructions:
    You are a customer service representative at a health insurance company and your job is to help providing the best accurate answer to a customer. 
    1. Limit your response to the information provided in this prompt. Do not conduct refer to external sources.
    2. Focus on the infromation rpesented here to form your answer.
    3. Do not include personal opinions or assumptions; stick to the facts provided in the prompt.
    4. Explain the answer with reference to the provided paragraphs. If the answer is not in the paragraphs, say that you do not know.
    5. Write your answer below the prompt.

    Duplicate anesthesia services submitted by the same physician (or physician group), different physician or other quali ed health care professional for the 

Would you like to ask a related question? Enter Y for yes and N for no. n


In [ ]:
generate_text("""Question: I haven't gotten my vaccines this year. Will I owe a copay if I get the flu shot?
Context: • Vaccines, including Pneumonia, Flu shots, Hepatitis B,
COVID-19 and other vaccines (counseling for people with no sign of tobacco-related
disease)
• Vaccines, including Pneumonia, Flu shots, Hepatitis B,
COVID-19 and other vaccines
• “Welcome to Medicare” preventive visit (one-time)
Any additional preventive services approved by Medicare during
the contract year will be covered. Flu shot, Hepatitis B,
Pneumonia, and COVID-19 vaccines are $0 copayment in-and
out-of-network. 

Important Message About What You Pay for Vaccines – Our
plan covers most Part D vaccines at no cost to you even if you
haven’t paid your deductible. Call Member Services for more
information.
Covered Benefits
NOTE: Services with a 1 may require prior authorization. 

$0 copayment
• Out-of-network
o $50 copayment in an office or freestanding facility
o $110 copayment in an outpatient hospital
Diagnostic Mammogram:
• In-network: $0 copayment
• Out-of-network:
o $60 copayment in an office or freestanding facility
o $175 copayment in an outpatient hospital
Diagnostic radiology services (such as MRIs, CT scans):
• In-network:
o $40 copayment in an office or freestanding facility
o $150 copayment in an outpatient hospital 
              """)

In [ ]:
generate_text("""Question: Are hearing aids covered under my plan?
Context: Your provider must submit claims to HearUSA
for any in-network and out-of-network routine hearing
exams.
Fitting/Evaluation for hearing aid (1 per year):
• In-network: $0 copayment
• Out-of-network: $30 copayment
• Call HearUSA to schedule a visit with an in-network
provider. Your provider must submit claims to HearUSA
for any in-network and out-of-network fitting/evaluation
for hearing aid.
Our plan covers up to $1,250 every year for hearing aids. Plan
covers $750 toward the purchase of a hearing aid for one ear
and $500 toward the purchase of a hearing aid for the second
ear. You are responsible for payment beyond the $1,250
coverage limit. One (1) year supply of batteries are included.

Covered Benefits Braven Medicare Choice (PPO)
NOTE: Services with a 1 may require prior authorization.
Hearing Services You can obtain hearing aids from any HearUSA in-network
provider at a discount. If you obtain hearing aids from an out-ofnetwork provider, submit your request to HearUSA for
reimbursement up to a $1,250 coverage limit.
Dental Services Routine dental services (preventive/diagnostic):

In-network: $20 copayment
• Out-of-network: $30 copayment
Routine hearing exam (1 per year):
• In-network: $0 copayment
• Out-of-network: $30 copayment
• Call HearUSA to schedule a visit with an in-network
provider. Your provider must submit claims to HearUSA
for any in-network and out-of-network routine hearing
exams.
Fitting/Evaluation for hearing aid (1 per year):
• In-network: $0 copayment
• Out-of-network: $30 copayment
• Call HearUSA to schedule a visit with an in-network
              """)

In [1]:
answer_question("Are their any limitations and exclusions?")

NameError: name 'answer_question' is not defined

In [ ]:
answer_question("Are their any limitations and exclusions?")